# Tarea 1

**Autor:** Alejandro Zarate Macias  
**Curso:** Reconocimiento de Patrones (ML)
**Fecha:** 02 de Febrero 2026

---

## Introducción
En este trabajo se abordan distintos problemas de regresión lineal y polinomial aplicados al reconocimiento de patrones. Se utilizan tanto conjuntos de datos reales como funciones analíticas para analizar el comportamiento de los modelos de regresión bajo diferentes configuraciones. Se estudia el efecto del grado del polinomio, el número de iteraciones y la inclusión de regularización L1 y L2. Finalmente, se emplean curvas de aprendizaje para evaluar el desempeño de los modelos y analizar fenómenos de subajuste y sobreajuste.

---

## Pre-requisitos
- Python 3.9 o superior
- Librerías:
    - numpy
    - matplotlib
    - notebook
    - pandas
    - scikit-learn
    - kagglehub[pandas-datasets]
    - ipywidgets

---

## Carga de Datos
Se utiliza el conjunto de datos `2022 Fuel Consumption Ratings` disponible en Kaggle. Para facilitar la descarga de estos mismos, se implementa la funcion `get_dataset` para ser reutilizada por los problemas en los que se requiera.

In [101]:
import pandas as pd
import kagglehub
from kagglehub import KaggleDatasetAdapter

def get_dataset() -> pd.DataFrame:
    file_path = "MY2022 Fuel Consumption Ratings.csv"

    df = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "rinichristy/2022-fuel-consumption-ratings",
    file_path
    )
    return df

---

# Problema 1

Considere el conjunto de datos de `2022 Fuel Consumption Ratings`. Elabore un script en Python para resolver el problema de regresión asociado con la predicción del consumo de combustible en ciudad y en carretera utilizando sklearn. Puede usar dos modelos lineales separados, uno para cada categoría de consumo de combustible. Escriba todas las suposiciones y las operaciones de preprocesamiento de datos que realice.

In [102]:
# Libreras necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [103]:
# Carga de datos y exploracion
df = get_dataset()
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 946 entries, 0 to 945
Data columns (total 15 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Model Year                         946 non-null    int64  
 1   Make                               946 non-null    object 
 2   Model                              946 non-null    object 
 3   Vehicle Class                      946 non-null    object 
 4   Engine Size(L)                     946 non-null    float64
 5   Cylinders                          946 non-null    int64  
 6   Transmission                       946 non-null    object 
 7   Fuel Type                          946 non-null    object 
 8   Fuel Consumption (City (L/100 km)  946 non-null    float64
 9   Fuel Consumption(Hwy (L/100 km))   946 non-null    float64
 10  Fuel Consumption(Comb (L/100 km))  946 non-null    float64
 11  Fuel Consumption(Comb (mpg))       946 non-null    int64  

,Model Year,Make,Model,Vehicle Class,Engine Size(L),Cylinders,Transmission,Fuel Type,Fuel Consumption (City (L/100 km),Fuel Consumption(Hwy (L/100 km)),Fuel Consumption(Comb (L/100 km)),Fuel Consumption(Comb (mpg)),CO2 Emissions(g/km),CO2 Rating,Smog Rating
0,2022,Acura,ILX,Compact,2.4,4,AM8,Z,9.9,7.0,8.6,33,200,6,3
1,2022,Acura,MDX SH-AWD,SUV: Small,3.5,6,AS10,Z,12.6,9.4,11.2,25,263,4,5
2,2022,Acura,RDX SH-AWD,SUV: Small,2.0,4,AS10,Z,11.0,8.6,9.9,29,232,5,6
3,2022,Acura,RDX SH-AWD A-SPEC,SUV: Small,2.0,4,AS10,Z,11.3,9.1,10.3,27,242,5,6
4,2022,Acura,TLX SH-AWD,Compact,2.0,4,AS10,Z,11.2,8.0,9.8,29,230,5,7


In [104]:
# Analisis de variables categóricas
# Se imprime el numero distinto de valores por cada variable categórica
# Esto con el fin de decidir que variables se pueden usar en el modelo sin aumentar demasiado la dimensionalidad

for column in df.select_dtypes(include=['object']).columns:
    print(f"Column: {column}")
    print(f"Número de valores distintos:", df[column].nunique())
    print()

Column: Make
Número de valores distintos: 39

Column: Model
Número de valores distintos: 715

Column: Vehicle Class
Número de valores distintos: 14

Column: Transmission
Número de valores distintos: 23

Column: Fuel Type
Número de valores distintos: 4



In [105]:
# Seleccion de features y variables objetivo

# X:
# Variables numericas: 'Engine Size(L)', 'Cylinders'
# Variables categoricas: 'Vehicle Class', 'Transmission', 'Fuel Type'
X = df[['Engine Size(L)', 'Cylinders', 
        'Vehicle Class', 'Transmission', 'Fuel Type']]

# Manejo de variables categoricas con get_dummies
X = pd.get_dummies(X, columns=['Vehicle Class', 'Transmission', 'Fuel Type'], drop_first=True)


# y:
# Para el modelo de consumo en ciudad
y_city = df['Fuel Consumption (City (L/100 km)']

# Para el modelo de consumo en carretera
y_hwy = df['Fuel Consumption(Hwy (L/100 km))']

In [106]:
# Split de datos en entrenamiento y prueba
X_train_city, X_test_city, y_train_city, y_test_city = train_test_split(X, y_city, test_size=0.2, random_state=14)
X_train_hwy, X_test_hwy, y_train_hwy, y_test_hwy = train_test_split(X, y_hwy, test_size=0.2, random_state=14)

In [107]:
# Modelo para consumo en ciudad
model_city = LinearRegression()
model_city.fit(X_train_city, y_train_city)

y_city_train_pred = model_city.predict(X_train_city)
y_city_test_pred = model_city.predict(X_test_city)

print("=== Modelo Ciudad ===")
print(f"Train - MSE: {mean_squared_error(y_train_city, y_city_train_pred):.4f}, R²: {r2_score(y_train_city, y_city_train_pred):.4f}")
print(f"Test  - MSE: {mean_squared_error(y_test_city, y_city_test_pred):.4f}, R²: {r2_score(y_test_city, y_city_test_pred):.4f}")

=== Modelo Ciudad ===
Train - MSE: 1.6035, R²: 0.8697
Test  - MSE: 1.3690, R²: 0.8663


In [108]:
# Modelo para consumo en carretera
model_hwy = LinearRegression()
model_hwy.fit(X_train_hwy, y_train_hwy)

y_hwy_train_pred = model_hwy.predict(X_train_hwy)
y_hwy_test_pred = model_hwy.predict(X_test_hwy)

print("=== Modelo Carretera===")
print(f"Train - MSE: {mean_squared_error(y_train_hwy, y_hwy_train_pred):.4f}, R²: {r2_score(y_train_hwy, y_hwy_train_pred):.4f}")
print(f"Test  - MSE: {mean_squared_error(y_test_hwy, y_hwy_test_pred):.4f}, R²: {r2_score(y_test_hwy, y_hwy_test_pred):.4f}")

=== Modelo Carretera===
Train - MSE: 1.0452, R²: 0.8084
Test  - MSE: 0.9527, R²: 0.7752
